In [75]:
import pandas as pd
import numpy as np
import requests
import os
import time
import math
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from datetime import datetime, timedelta


# Add industries

In [425]:
seed_list = pd.read_excel('Company seed list with categories.xlsx')

In [426]:
seed_list['Company'] = seed_list['Company'].apply(lambda x: x.split('\n')[0])

In [427]:
seed_list_scraping = seed_list.drop_duplicates(subset = ['Company']).reset_index(drop=True)

In [428]:
seed_list_scraping[seed_list_scraping.Company=='Booking Holdings (Booking.com)']

,Company,Market_Cap_URL,Listing,Glassdoor ID,Categories
21,Booking Holdings (Booking.com),https://companiesmarketcap.com/booking-holding...,Top 100 US,256653,Tech


In [429]:
seed_list = seed_list.drop_duplicates(subset = ['Company']).reset_index(drop=True)

In [430]:
seed_list.drop(columns=['Market_Cap_URL','Listing','Glassdoor ID'], inplace=True)

In [431]:
seed_list.columns = ['Company', 'Industry']

In [432]:
seed_list

,Company,Industry
0,Abbott Laboratories,"Pharmaceuticals, Healthcare, Medical devices"
1,AbbVie,"Pharmaceuticals, Healthcare, Medical devices"
2,Adobe,Tech
3,Airbnb,Tech
4,Alphabet (Google),Tech
...,...,...
100,Cadence Design Systems,Tech
101,Dell,Tech
102,Equinix,Tech
103,Fiserv,Tech


In [121]:
categories = pd.read_excel('Categories.xlsx')

In [127]:
categories = categories[categories.Categories!='🇺🇸 Dow jones']

In [129]:
categories = categories.groupby('Market_Cap_URL')['Categories'].first().reset_index()

In [131]:
seed_list = seed_list.merge(categories, on='Market_Cap_URL', how='left')

# Develop the scraper

## Example

In [33]:
# JS

url = "https://www.glassdoor.com/graph"

payload = [
    {
        "operationName": "RecordPageView",
        "variables": {
            "employerId": "11159",
            "metaData": [],
            "pageIdent": "INFOSITE_REVIEWS"
        },
        "query": "mutation RecordPageView($employerId: String!, $pageIdent: String!) {recordPageView(pageIdent: $pageIdent metaData: {key: \"employerId\", value: $employerId}) {totalCount  __typename  }"
    },
    {
        "operationName": "GetEmployerReviews",
        "variables": {
            "applyDefaultCriteria": True,
            "employerId": 11159,
            "employmentStatuses": [],
            "goc": None,
            "jobTitle": None,
            "language": "eng",
            "location": {
                "countryId": None,
                "stateId": None,
                "metroId": None,
                "cityId": None
            },
            "locationName": "",
            "mlHighlightSearch": None,
            "onlyCurrentEmployees": False,
            "page": 2,
            "preferredTldId": 0,
            "reviewCategories": [],
            "sort": "RELEVANCE",
            "textSearch": "",
            "worldwideFilter": False,
            "dynamicProfileId": 919129,
            "useRowProfileTldForRatings": True,
            "enableKeywordSearch": False
        },
        "query": "query GetEmployerReviews($applyDefaultCriteria: Boolean, $dynamicProfileId: Int, $employerId: Int!, $employmentStatuses: [EmploymentStatusEnum], $enableKeywordSearch: Boolean!, $goc: GOCIdent, $isRowProfileEnabled: Boolean, $jobTitle: JobTitleIdent, $language: String, $location: LocationIdent, $onlyCurrentEmployees: Boolean, $page: Int!, $preferredTldId: Int, $reviewCategories: [ReviewCategoriesEnum], $sort: ReviewsSortOrderEnum, $textSearch: String, $useRowProfileTldForRatings: Boolean, $worldwideFilter: Boolean) {employerReviews: employerReviewsRG( employerReviewsInput: {applyDefaultCriteria: $applyDefaultCriteria, dynamicProfileId: $dynamicProfileId, employer: {id: $employerId}, employmentStatuses: $employmentStatuses, onlyCurrentEmployees: $onlyCurrentEmployees, goc: $goc, isRowProfileEnabled: $isRowProfileEnabled, jobTitle: $jobTitle, language: $language, location: $location, page: {num: $page, size: 10}, preferredTldId: $preferredTldId, reviewCategories: $reviewCategories, sort: $sort, textSearch: $textSearch, useRowProfileTldForRatings: $useRowProfileTldForRatings, worldwideFilter: $worldwideFilter}) {allReviewsCount currentPage filteredReviewsCount   lastReviewDateTime   numberOfPages queryJobTitle {   id    text     __typename   }  queryLocation {     id     longName    shortName      type    __typename} ratedReviewsCount ratings {   businessOutlookRating     careerOpportunitiesRating   ceoRating    compensationAndBenefitsRating    cultureAndValuesRating  diversityAndInclusionRating     overallRating  ratedCeo {      id      largePhoto: photoUrl(size: LARGE)      name      regularPhoto: photoUrl(size: REGULAR)       title      __typename    }    recommendToFriendRating    reviewCount    seniorManagementRating    workLifeBalanceRating     __typename    }  reviews {advice     adviceOriginal    cons     consOriginal     countHelpful    countNotHelpful    employer {      id       largeLogoUrl: squareLogoUrl(size: LARGE)       regularLogoUrl: squareLogoUrl(size: REGULAR)        shortName       __typename     }     employerResponses {      id       countHelpful        countNotHelpful       languageId       originalLanguageId       response       responseDateTime(format: ISO)      responseOriginal       translationMethod       __typename      }     employmentStatus    featured     isCurrentJob    jobTitle {     id       text       __typename     }     languageId     lengthOfEmployment    location {       id       type       name      __typename     }     originalLanguageId     pros      prosOriginal      ratingBusinessOutlook      ratingCareerOpportunities     ratingCeo    ratingCompensationAndBenefits      ratingCultureAndValues     ratingDiversityAndInclusion     ratingOverall      ratingRecommendToFriend    ratingSeniorLeadership    ratingWorkLifeBalance      reviewDateTime    reviewId      summary     summaryOriginal     textSearchHighlightPhrases @include(if: $enableKeywordSearch) {        field       phrases {        length          position: pos        __typename       }       __typename     }     translationMethod    __typename    }   ratingCountDistribution {     overall {       _5       _4       _3       _2       _1      __typename     }     cultureAndValues {      _5       _4       _3        _2      _1      __typename    }     careerOpportunities {       _5       _4       _3      _2       _1       __typename     }      workLifeBalance {        _5       _4       _3       _2       _1       __typename     }     seniorManagement {       _5       _4       _3       _2      _1      __typename     }      compensationAndBenefits {       _5       _4       _3       _2      _1       __typename     }      diversityAndInclusion {        _5        _4       _3      _2       _1        __typename     }      recommendToFriend {        WONT_RECOMMEND     RECOMMEND      __typename   }    __typename   }   __typename}}"
    }
]
headers = {
    "authority": "www.glassdoor.com",
    "accept": "*/*",
    "apollographql-client-name": "ei-reviews-next",
    "apollographql-client-version": "1.27.13",
    "content-type": "application/json",
    "origin": "https://www.glassdoor.com",
    "referer": "https://www.glassdoor.com/",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)


[{"errors":[{"message":"Syntax Error: Expected Name, found <EOF>.","locations":[{"line":1,"column":183}],"extensions":{"code":"GRAPHQL_PARSE_FAILED"}}]}
,{"data":{"employerReviews":{"allReviewsCount":19696,"currentPage":2,"filteredReviewsCount":17093,"lastReviewDateTime":"2024-03-23T15:02:10.757","numberOfPages":1710,"queryJobTitle":null,"queryLocation":null,"ratedReviewsCount":17986,"ratings":{"businessOutlookRating":0.71,"careerOpportunitiesRating":3.9,"ceoRating":0.8,"compensationAndBenefitsRating":4.3,"cultureAndValuesRating":4,"diversityAndInclusionRating":4.3,"overallRating":4.1,"ratedCeo":{"id":568377,"largePhoto":"https://media.glassdoor.com/people/sqll/11159/salesforce-ceo1684713833445.png","name":"Marc Benioff","regularPhoto":"https://media.glassdoor.com/people/sql/11159/salesforce-ceo1684713833445.png","title":"CEO ","__typename":"Ceo"},"recommendToFriendRating":0.79,"reviewCount":17986,"seniorManagementRating":3.6,"workLifeBalanceRating":3.9,"__typename":"EmployerRatings"},

In [116]:
# Mine 
url = "https://www.glassdoor.com/graph"

payload = [
    {
        "operationName":"RecordPageView",
        "variables":{
            "employerId": "1557", #"1138",
            "metaData":[],
            "pageIdent":"INFOSITE_REVIEWS"},
        "query":"mutation RecordPageView($employerId: String!, $pageIdent: String!) {\n  recordPageView(\n    pageIdent: $pageIdent\n    metaData: {key: \"employerId\", value: $employerId}\n  ) {\n    totalCount\n    __typename\n  }\n}\n".replace('\n', '')},
    {"operationName":"GetEmployerReviews",
     "variables":{"applyDefaultCriteria":False,
                  "employerId":1557, #1138
                  "employmentStatuses":["REGULAR","PART_TIME"],
                  "goc":None,
                  "jobTitle":None,
                  "language":"eng",
                  "location":{"countryId":1,
                              "stateId":None,
                              "metroId":None,
                              "cityId":None},
                  "locationName":"US",
                  "mlHighlightSearch":None,
                  "onlyCurrentEmployees":False,
                  "page":0,
                  "preferredTldId":0,
                  "reviewCategories":[],
                  "sort":"DATE",
                  "textSearch":"",
                  "worldwideFilter":False,
                  "dynamicProfileId":1140,
                  "useRowProfileTldForRatings":False,
                  "enableKeywordSearch":False},
     "query":"query GetEmployerReviews($applyDefaultCriteria: Boolean, $dynamicProfileId: Int, $employerId: Int!, $employmentStatuses: [EmploymentStatusEnum], $enableKeywordSearch: Boolean!, $goc: GOCIdent, $isRowProfileEnabled: Boolean, $jobTitle: JobTitleIdent, $language: String, $location: LocationIdent, $onlyCurrentEmployees: Boolean, $page: Int!, $preferredTldId: Int, $reviewCategories: [ReviewCategoriesEnum], $sort: ReviewsSortOrderEnum, $textSearch: String, $useRowProfileTldForRatings: Boolean, $worldwideFilter: Boolean) {\n  employerReviews: employerReviewsRG(\n    employerReviewsInput: {applyDefaultCriteria: $applyDefaultCriteria, dynamicProfileId: $dynamicProfileId, employer: {id: $employerId}, employmentStatuses: $employmentStatuses, onlyCurrentEmployees: $onlyCurrentEmployees, goc: $goc, isRowProfileEnabled: $isRowProfileEnabled, jobTitle: $jobTitle, language: $language, location: $location, page: {num: $page, size: 1000}, preferredTldId: $preferredTldId, reviewCategories: $reviewCategories, sort: $sort, textSearch: $textSearch, useRowProfileTldForRatings: $useRowProfileTldForRatings, worldwideFilter: $worldwideFilter}\n  ) {\n    allReviewsCount\n    currentPage\n    filteredReviewsCount\n    lastReviewDateTime\n    numberOfPages\n    queryJobTitle {\n      id\n      text\n      __typename\n    }\n    queryLocation {\n      id\n      longName\n      shortName\n      type\n      __typename\n    }\n    ratedReviewsCount\n    ratings {\n      businessOutlookRating\n      careerOpportunitiesRating\n      ceoRating\n      compensationAndBenefitsRating\n      cultureAndValuesRating\n      diversityAndInclusionRating\n      overallRating\n      ratedCeo {\n        id\n        largePhoto: photoUrl(size: LARGE)\n        name\n        regularPhoto: photoUrl(size: REGULAR)\n        title\n        __typename\n      }\n      recommendToFriendRating\n      reviewCount\n      seniorManagementRating\n      workLifeBalanceRating\n      __typename\n    }\n    reviews {\n      advice\n      adviceOriginal\n      cons\n      consOriginal\n      countHelpful\n      countNotHelpful\n      employer {\n        id\n        largeLogoUrl: squareLogoUrl(size: LARGE)\n        regularLogoUrl: squareLogoUrl(size: REGULAR)\n        shortName\n        __typename\n      }\n      employerResponses {\n        id\n        countHelpful\n        countNotHelpful\n        languageId\n        originalLanguageId\n        response\n        responseDateTime(format: ISO)\n        responseOriginal\n        translationMethod\n        __typename\n      }\n      employmentStatus\n      featured\n      isCurrentJob\n      jobTitle {\n        id\n        text\n        __typename\n      }\n      languageId\n      lengthOfEmployment\n      location {\n        id\n        type\n        name\n        __typename\n      }\n      originalLanguageId\n      pros\n      prosOriginal\n      ratingBusinessOutlook\n      ratingCareerOpportunities\n      ratingCeo\n      ratingCompensationAndBenefits\n      ratingCultureAndValues\n      ratingDiversityAndInclusion\n      ratingOverall\n      ratingRecommendToFriend\n      ratingSeniorLeadership\n      ratingWorkLifeBalance\n      reviewDateTime\n      reviewId\n      summary\n      summaryOriginal\n      textSearchHighlightPhrases @include(if: $enableKeywordSearch) {\n        field\n        phrases {\n          length\n          position: pos\n          __typename\n        }\n        __typename\n      }\n      translationMethod\n      __typename\n    }\n    ratingCountDistribution {\n      overall {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      cultureAndValues {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      careerOpportunities {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      workLifeBalance {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      seniorManagement {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      compensationAndBenefits {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      diversityAndInclusion {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      recommendToFriend {\n        WONT_RECOMMEND\n        RECOMMEND\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n".replace('\n', '')
    }]






response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

[{"data":{"recordPageView":{"totalCount":1,"__typename":"PageViewSummary"}}}
,{"data":{"employerReviews":{"allReviewsCount":1738,"currentPage":0,"filteredReviewsCount":639,"lastReviewDateTime":"2024-03-23T14:08:11.480","numberOfPages":1,"queryJobTitle":null,"queryLocation":{"id":1,"longName":"United States of America","shortName":"US","type":"COUNTRY","__typename":"Country"},"ratedReviewsCount":639,"ratings":{"businessOutlookRating":0.81,"careerOpportunitiesRating":3.8,"ceoRating":0.87,"compensationAndBenefitsRating":3.6,"cultureAndValuesRating":3.9,"diversityAndInclusionRating":3.9,"overallRating":4.1,"ratedCeo":{"id":65676,"largePhoto":"https://media.glassdoor.com/people/sqll/1557/kla-ceo1671041120103.png","name":"Rick Wallace","regularPhoto":"https://media.glassdoor.com/people/sql/1557/kla-ceo1671041120103.png","title":"President and Chief Executive Officer","__typename":"Ceo"},"recommendToFriendRating":0.83,"reviewCount":639,"seniorManagementRating":3.6,"workLifeBalanceRating":3.4,

## Scraper

In [100]:
headers = {
    "apollographql-client-name": "ei-reviews-next",
    "content-type": "application/json",
    "origin": "https://www.glassdoor.com",
    "referer": "https://www.glassdoor.com/",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
}

In [377]:
for index, row in seed_list_scraping.iloc[21:22].iterrows():
    
    company = row['Company']
    glassdoor_id = int(row['Glassdoor ID'])
    page_number = 0
    current_date = datetime.now()
    
    while current_date >= two_years_ago: 
    
        payload = [
        {
            "operationName":"RecordPageView",
            "variables":{
                "employerId": f"{glassdoor_id}", #Glassdoor ID
                "metaData":[],
                "pageIdent":"INFOSITE_REVIEWS"},
            "query":"mutation RecordPageView($employerId: String!, $pageIdent: String!) {\n  recordPageView(\n    pageIdent: $pageIdent\n    metaData: {key: \"employerId\", value: $employerId}\n  ) {\n    totalCount\n    __typename\n  }\n}\n".replace('\n', '')},
        {"operationName":"GetEmployerReviews",
         "variables":{"applyDefaultCriteria":False,
                      "employerId":glassdoor_id, # Glassdoor ID
                      "employmentStatuses":["REGULAR","PART_TIME"],
                      "goc":None,
                      "jobTitle":None,
                      "language":"eng",
                      "location":{"countryId":1,
                                  "stateId":None,
                                  "metroId":None,
                                  "cityId":None},
                      "locationName":"US",
                      "mlHighlightSearch":None,
                      "onlyCurrentEmployees":False,
                      "page":page_number,
                      "preferredTldId":0,
                      "reviewCategories":[],
                      "sort":"DATE",
                      "textSearch":"",
                      "worldwideFilter":False,
                      "dynamicProfileId":1140,
                      "useRowProfileTldForRatings":False,
                      "enableKeywordSearch":False},
         "query":"query GetEmployerReviews($applyDefaultCriteria: Boolean, $dynamicProfileId: Int, $employerId: Int!, $employmentStatuses: [EmploymentStatusEnum], $enableKeywordSearch: Boolean!, $goc: GOCIdent, $isRowProfileEnabled: Boolean, $jobTitle: JobTitleIdent, $language: String, $location: LocationIdent, $onlyCurrentEmployees: Boolean, $page: Int!, $preferredTldId: Int, $reviewCategories: [ReviewCategoriesEnum], $sort: ReviewsSortOrderEnum, $textSearch: String, $useRowProfileTldForRatings: Boolean, $worldwideFilter: Boolean) {\n  employerReviews: employerReviewsRG(\n    employerReviewsInput: {applyDefaultCriteria: $applyDefaultCriteria, dynamicProfileId: $dynamicProfileId, employer: {id: $employerId}, employmentStatuses: $employmentStatuses, onlyCurrentEmployees: $onlyCurrentEmployees, goc: $goc, isRowProfileEnabled: $isRowProfileEnabled, jobTitle: $jobTitle, language: $language, location: $location, page: {num: $page, size: 1000}, preferredTldId: $preferredTldId, reviewCategories: $reviewCategories, sort: $sort, textSearch: $textSearch, useRowProfileTldForRatings: $useRowProfileTldForRatings, worldwideFilter: $worldwideFilter}\n  ) {\n    allReviewsCount\n    currentPage\n    filteredReviewsCount\n    lastReviewDateTime\n    numberOfPages\n    queryJobTitle {\n      id\n      text\n      __typename\n    }\n    queryLocation {\n      id\n      longName\n      shortName\n      type\n      __typename\n    }\n    ratedReviewsCount\n    ratings {\n      businessOutlookRating\n      careerOpportunitiesRating\n      ceoRating\n      compensationAndBenefitsRating\n      cultureAndValuesRating\n      diversityAndInclusionRating\n      overallRating\n      ratedCeo {\n        id\n        largePhoto: photoUrl(size: LARGE)\n        name\n        regularPhoto: photoUrl(size: REGULAR)\n        title\n        __typename\n      }\n      recommendToFriendRating\n      reviewCount\n      seniorManagementRating\n      workLifeBalanceRating\n      __typename\n    }\n    reviews {\n      advice\n      adviceOriginal\n      cons\n      consOriginal\n      countHelpful\n      countNotHelpful\n      employer {\n        id\n        largeLogoUrl: squareLogoUrl(size: LARGE)\n        regularLogoUrl: squareLogoUrl(size: REGULAR)\n        shortName\n        __typename\n      }\n      employerResponses {\n        id\n        countHelpful\n        countNotHelpful\n        languageId\n        originalLanguageId\n        response\n        responseDateTime(format: ISO)\n        responseOriginal\n        translationMethod\n        __typename\n      }\n      employmentStatus\n      featured\n      isCurrentJob\n      jobTitle {\n        id\n        text\n        __typename\n      }\n      languageId\n      lengthOfEmployment\n      location {\n        id\n        type\n        name\n        __typename\n      }\n      originalLanguageId\n      pros\n      prosOriginal\n      ratingBusinessOutlook\n      ratingCareerOpportunities\n      ratingCeo\n      ratingCompensationAndBenefits\n      ratingCultureAndValues\n      ratingDiversityAndInclusion\n      ratingOverall\n      ratingRecommendToFriend\n      ratingSeniorLeadership\n      ratingWorkLifeBalance\n      reviewDateTime\n      reviewId\n      summary\n      summaryOriginal\n      textSearchHighlightPhrases @include(if: $enableKeywordSearch) {\n        field\n        phrases {\n          length\n          position: pos\n          __typename\n        }\n        __typename\n      }\n      translationMethod\n      __typename\n    }\n    ratingCountDistribution {\n      overall {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      cultureAndValues {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      careerOpportunities {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      workLifeBalance {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      seniorManagement {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      compensationAndBenefits {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      diversityAndInclusion {\n        _5\n        _4\n        _3\n        _2\n        _1\n        __typename\n      }\n      recommendToFriend {\n        WONT_RECOMMEND\n        RECOMMEND\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n".replace('\n', '')
        }]

        try:
            response = requests.request("POST", url, json=payload, headers=headers)
            response.raise_for_status()
            time.sleep(max(25.564878, 26.091354 + 2*np.random.randn()))

            json_company = response.json()
            file_path = os.path.join(folder_path, f'{company}_{page_number}.json')

                # Save the JSON data 
            with open(file_path, 'w', encoding='utf-8') as json_file:
                json.dump(json_company, json_file, ensure_ascii=False, indent=4)

            print('Done row ', index, ' ', company, page_number)
            page_number +=1
            try:
                current_date = datetime.strptime(json_company[1]['data']['employerReviews']['reviews'][-1]['reviewDateTime'], '%Y-%m-%dT%H:%M:%S.%f')

            except:
                current_date = datetime.strptime(json_company[1]['data']['employerReviews']['reviews'][-1]['reviewDateTime'], '%Y-%m-%dT%H:%M:%S')

        except:
            print('Exception: ', index, ' ', company, page_number)
            time.sleep(max(5.564878, 10.091354 + 2*np.random.randn()))
        

Done row  21   Booking Holdings (Booking.com) 0


# Read scraped data

In [398]:
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

In [399]:
data = []

In [400]:
for file in json_files:
    with open(os.path.join(folder_path, file), 'r') as json_file:
        json_data = json.load(json_file)
        company_name = file.split('.json')[0].split('_')[0]
        reviews = json_data[1]['data']['employerReviews']['reviews']
        for review in reviews:
            reviewId = review.get('reviewId')
            ratingOverall = review.get('ratingOverall')
            location = review.get('location').get('name')
            ratingBusinessOutlook = review.get('ratingBusinessOutlook')
            reviewDateTime = review.get('reviewDateTime')
            
            data.append({'review_id': reviewId, 'rating_overall': ratingOverall,
                    'location': location, 'business_outlook': ratingBusinessOutlook,
                        'review_date': reviewDateTime, 'Company': company_name})
        

In [624]:
df = pd.DataFrame(data)

In [625]:
df = df.drop_duplicates('review_id')

In [626]:
df['review_date'] = df['review_date'].apply(lambda x: datetime.strptime(x.split('T')[0], '%Y-%m-%d'))

In [627]:
current_date = datetime.strptime('2024-03-23', '%Y-%m-%d')
two_years_ago = current_date - timedelta(days=365 * 2) 

In [628]:
one_year_ago = current_date - timedelta(days=365) 

In [629]:
df = df[df['review_date'] >= two_years_ago].reset_index(drop=True)

In [630]:
abbreviations = pd.read_excel('/Users/anna.zemit/Desktop/Programming/2023/09. The Highest-Rated Independent Restaurants in Every State/State and Territory Abbreviations.xlsx')

In [631]:
for i in df['location']:
    try:
        i.split(',')[-1].strip()
    except:
        print(i)

In [632]:
df['Postal'] = df['location'].apply(lambda x: x.split(',')[-1].strip())

In [633]:
city_to_state = {
    'Chicago Hts': 'IL',  # Assuming Chicago Hts refers to Chicago Heights, IL
    #'Manatí': 'PR',  # Assuming Manatí is in Puerto Rico
    'Tri-Cities': 'WA',  # Assuming Tri-Cities refers to cities in Washington State
    'Los Angeles': 'CA',
    'Long Island-Queens': 'NY',  # Assuming Long Island-Queens refers to New York
    'San Tan Valley': 'AZ',
    'Minneapolis-Saint Paul': 'MN',
    'San Diego Country Estates': 'CA',
    'Las Vegas': 'NV',
    #'Gold Coast': 'QLD',  # Assuming Gold Coast refers to Queensland, Australia
    'Las Vegas Valley': 'NV',
    'Remote': None,  # Assuming Remote means outside any specific state or country
    'Midtown New York': 'NY',
    'Sacramento Valley': 'CA',
    'SoHo': 'NY',
    'San Antonio': 'TX',
    #'Las Piñas': 'PH',  # Assuming Las Piñas refers to the city in the Philippines
    #'Australia Island': 'AU',  # Assuming Australia Island refers to the whole country
    'Sacramento Canyon': 'CA',
    'Vegas': 'NV',
    'Emerson Hill': 'NY',
    'Island of Hawai‘i': 'HI',
    
}

# Define a dictionary to map city names to state codes
city_to_state.update({
    'Midtown New York': 'NY',
    'Sacramento Valley': 'CA',
    'SoHo': 'NY',
    'San Antonio': 'TX',
    'Sacramento Canyon': 'CA',
    'Vegas': 'NV',
    'Emerson Hill': 'NY',
    'Island of Hawai‘i': 'HI',
    'North Chesterfield': 'VA',
    'Massachusetts Avenue Heights': 'MA',
    'Lansdowne-Baltimore Highlands': 'MD',
    'San Jose': 'CA',
    'Jurupa Valley': 'CA',
    'Nashville-Davidson': 'TN',
    'Miami Township': 'OH',
    'Brooklyn Heights': 'NY',
    'Oahu Island': 'HI',
    'Irvine': 'CA',
    'Inland Empire': 'CA',
    'Bushwick': 'NY',
    'LaGrange': 'GA',
    'Shelby Charter Township': 'MI',
    'Mission Valley': 'CA',
    'Township of Ocean': 'NJ',
    'Connecticut Park': 'CT',
    'Honolulu District': 'HI',
    'Dupont Circle': 'DC',
    'Texarkana': 'TX',
    #'St. John': 'VI',  # Assuming St. John refers to the Virgin Islands
    'Lookout Valley': 'TN',
    'Brickell Point': 'FL',
})

city_to_state.update({
    'Sheepshead Bay': 'NY',
    'Town of Perinton': 'NY',
    'Town of Clarkstown': 'NY',
    'Township of Millcreek': 'PA',
    'Powdersville': 'SC',
    'Harlem': 'NY',
    'Eltingville': 'NY',
    'Otay Mesa': 'CA',
    'Chicago Lawn': 'IL',
    'Cotswold': 'NC',
    'Virgin Island': 'VI',
    'Kearny Mesa': 'CA',
    'Scottsdale': 'AZ',
    'Salt Lake International Center': 'UT',
    'Saratoga': 'CA',
    'Campo Rico': 'PR',
    'Union City': 'CA',
    'Porters Neck': 'NC',
    "Hell's Kitchen": 'NY',
    'Menominee': 'MI',
    'Candelaria': 'PR',
    'Los Feliz': 'CA',
    'Manhattan': 'NY',
    'Upper West Side': 'NY',
    'Santa Clara Valley': 'CA',
    'Pilsen': 'IL',
    'San Fernando Valley': 'CA',
    'Utahna': 'UT',
    'Township of Lawrence': 'NJ',
    'Lincoln Park': 'IL',
    'Cherry Hill Mall': 'NJ',
})

city_to_state.update({
    'United States Coast Guard - Air Station Sacramento': 'CA',
    'Eastvale': 'CA',
    'Dallas-Fort Worth': 'TX',
    'Oyster Point': 'VA',
    'Lemmon Valley-Golden Valley': 'NV',
    'Moorestown-Lenola': 'NJ',
    'Patuxent River Naval Air Station': 'MD',
    'Wallingford Center': 'CT',
    'Seacliff': 'NY',
    'Capitol Hill': 'DC',
    'Brookpark Estates': 'OH',
    'Steinway': 'NY',
    'Wall Street': 'NY',
    'Kent Island': 'MD',
    'Peapack and Gladstone': 'NJ',
    'Chula Vista-Orason': 'CA',
    'Queen Anne': 'WA',
    'Coconut Point': 'FL',
    'Pryor Creek': 'OK',
    'Ft. Worth Naval Air Station Joint Reserve Base': 'TX',
    #'Brasilia': 'BR',
    'Berkley Heights': 'NJ',
    'Tucson Estates': 'AZ',
    'Summerlin': 'NV',
    #'Bowmanville': 'ON',
    'Alderwood': 'WA',
    'Greenburgh': 'NY',
    'Los Angeles Subdivision': 'CA',
    'Boca Chica': 'TX'
})
city_to_state.update({
    'Temecula Valley': 'CA',
    'Bay Terrace': 'NY',
    'St. Charles': 'IL',
    'Lynhaven': 'VA',
    'Oxon Hill-Glassmanor': 'MD',
    'Bay Ridge': 'NY',
    'James Island': 'SC',
    'Georgetown': 'DC',
    'Fort Devens': 'MA',
    'WDM': 'IA',
    'Greenbriar': 'VA',
    'Dulles Town Center': 'VA',
    'Lansdale': 'PA',
    'Quad Cities': 'IA',
    'Tinicum Township': 'PA',
    'Ocracoke Island': 'NC',
    'Robinson Township': 'PA',
    #'Venezuela': 'VE',
    'East Honolulu': 'HI',
    'Camp Creek Estates': 'GA',
    'Barrigada Heights': 'GU',
    'Yakima Valley': 'WA',
    'Chula Vista': 'CA',
    'Township of Hamilton': 'NJ',
    'Outer Banks': 'NC',
    'St. James': 'NY',
    'Fruitridge Pocket': 'CA',
    'Koreatown': 'CA',
    'Hog Island': 'VA',
    "Martha's Vineyard": 'MA',
    'Arlington Hts': 'IL',
    'San Ramon Valley': 'CA',
    'Sheridan': 'WY'
})

In [634]:
df['Postal'] = df['Postal'].replace(city_to_state)

In [635]:
drop_locations = ['Manatí', 'Las Piñas', 'Australia Island', 'QLD', 'St. John',
                 'GU','AS', 'Brasilia','Bowmanville', 'Venezuela', 'PR', 'VI']

In [636]:
df = df[~df['Postal'].isin(drop_locations)]

In [637]:
len(df['Postal'].unique())

53

In [638]:
df[df['Postal']=='Gold Coast']

,review_id,rating_overall,location,business_outlook,review_date,Company,Postal
5386,78630117,4,Gold Coast,POSITIVE,2023-07-27,JPMorgan Chase,Gold Coast


In [639]:
df['Postal'].unique()

array(['FL', 'CA', 'TX', 'TN', 'KY', 'DE', 'IL', 'DC', 'VA', 'WA', 'IN',
       'CT', 'NV', 'CO', 'OH', 'NC', 'WI', 'MD', 'NE', 'OR', 'GA', 'NJ',
       'MI', 'AZ', 'PA', 'KS', 'NY', 'ID', 'MA', 'AL', 'MN', 'MO', 'OK',
       'UT', 'SC', 'MS', 'NM', 'IA', 'ND', 'MT', 'AR', 'RI', 'ME', 'NH',
       'LA', 'WV', 'VT', 'SD', 'Gold Coast', None, 'WY', 'AK', 'HI'],
      dtype=object)

In [641]:
df = df.merge(abbreviations, on='Postal', how='left')

In [642]:
df.State.unique()

array(['Florida', 'California', 'Texas', 'Tennessee', 'Kentucky',
       'Delaware', 'Illinois', 'District of Columbia', 'Virginia',
       'Washington', 'Indiana', 'Connecticut', 'Nevada', 'Colorado',
       'Ohio', 'North Carolina', 'Wisconsin', 'Maryland', 'Nebraska',
       'Oregon', 'Georgia', 'New Jersey', 'Michigan', 'Arizona',
       'Pennsylvania', 'Kansas', 'New York', 'Idaho', 'Massachusetts',
       'Alabama', 'Minnesota', 'Missouri', 'Oklahoma', 'Utah',
       'South Carolina', 'Mississippi', 'New Mexico', 'Iowa',
       'North Dakota', 'Montana', 'Arkansas', 'Rhode Island', 'Maine',
       'New Hampshire', 'Louisiana', 'West Virginia', 'Vermont',
       'South Dakota', nan, 'Wyoming', 'Alaska', 'Hawaii'], dtype=object)

In [643]:
len(df.State.unique())

52

In [644]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198617 entries, 0 to 198616
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   review_id         198617 non-null  int64         
 1   rating_overall    198617 non-null  int64         
 2   location          198617 non-null  object        
 3   business_outlook  110573 non-null  object        
 4   review_date       198617 non-null  datetime64[ns]
 5   Company           198617 non-null  object        
 6   Postal            198560 non-null  object        
 7   State             198559 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 13.6+ MB


In [645]:
df.drop(columns=['location', 'Postal'], inplace=True)

In [646]:
df = df.merge(seed_list, on='Company', how='left')

In [647]:
df

,review_id,rating_overall,business_outlook,review_date,Company,State,Industry
0,79012884,4,None,2023-08-09,Amazon,Florida,Tech
1,79011408,4,NEUTRAL,2023-08-09,Amazon,California,Tech
2,79011236,1,NEUTRAL,2023-08-09,Amazon,Texas,Tech
3,79010569,4,POSITIVE,2023-08-09,Amazon,Tennessee,Tech
4,79010303,1,NEGATIVE,2023-08-09,Amazon,California,Tech
...,...,...,...,...,...,...,...
198612,61759513,2,None,2022-03-25,Oracle,Texas,Tech
198613,61741825,5,POSITIVE,2022-03-24,Oracle,California,Tech
198614,61733583,3,None,2022-03-24,Oracle,California,Tech
198615,61718820,5,POSITIVE,2022-03-24,Oracle,Texas,Tech


# Analysis

In [327]:
import os

# Path to the folder containing the .webp images

# List all files in the folder
file_list = os.listdir(folder_path)

# Iterate through each file and rename it
for filename in file_list:
    # Check if the file is a .webp image
    if filename.endswith(".webp"):
        # Extract the new filename by removing the prefix and suffix
        new_filename = filename.replace("https___companiesmarketcap.com_", "").replace("_marketcap_.webp", ".webp")
        
        # Construct the old and new file paths
        old_filepath = os.path.join(folder_path, filename)
        new_filepath = os.path.join(folder_path, new_filename)
        
        # Rename the file
        os.rename(old_filepath, new_filepath)


 Rank companies overall and per industry on the percentage of employees with positive business outlook for the most recent 12 months

In [648]:
number_of_reviews = df.groupby('Company')['review_id'].count().reset_index()
number_of_reviews.columns = ['Company', 'reviews_total']
number_of_reviews

,Company,reviews_total
0,AMD,260
1,AT&T,4422
2,AbbVie,620
3,Abbott Laboratories,890
4,Adobe,550
...,...,...
99,Vertex Pharmaceuticals,209
100,Visa,465
101,Walmart,17431
102,Walt Disney,1907


In [649]:
number_of_reviews_last_year = df[df.review_date >= one_year_ago].groupby('Company')['review_id'].count().reset_index()
number_of_reviews_last_year.columns = ['Company', 'reviews_total']

In [650]:
len(df[df.review_date >= one_year_ago])

112804

In [651]:
number_of_reviews_two_years_ago =  df[df.review_date < one_year_ago].groupby('Company')['review_id'].count().reset_index()
number_of_reviews_two_years_ago.columns = ['Company', 'reviews_total']

In [652]:
len(df[df.review_date <= one_year_ago])

86153

In [653]:
quantile_25_two_years_ago = number_of_reviews_two_years_ago.reviews_total.quantile(0.25)
quantile_25_two_years_ago

122.0

In [654]:
quantile_25_last_year = number_of_reviews_last_year.reviews_total.quantile(0.25)
quantile_25_last_year

170.75

In [655]:
(quantile_25_last_year + quantile_25_two_years_ago)/2

146.375

In [656]:
quantile_25 = number_of_reviews.reviews_total.quantile(0.25)
quantile_25

283.25

# Organise Tabs and Rankings

## Top 10

Top 10: Top 10 companies on % of employees with positive business outlook

Columns: Company, % of employees with positive business outlook, sample size of employees, company logo image link


In [506]:
#top_10 = df[df.review_date >= one_year_ago]

In [657]:
top_10 = df[df.business_outlook=='POSITIVE'].groupby('Company')['review_id'].count().reset_index()

In [658]:
top_10.columns = ['Company', 'Number of Positive reviews']

In [659]:
top_10 = top_10.merge(number_of_reviews, on='Company', how='left')

In [660]:
top_10['positive business outlook %'] = top_10['Number of Positive reviews']/top_10['reviews_total']

In [661]:
top_10 = top_10[top_10.reviews_total >=quantile_25]

In [662]:
top_10 = top_10.sort_values(by='positive business outlook %', ascending=False).head(10)

In [663]:
top_10['company logo image name'] = top_10['Company'].apply(lambda x: x.lower().replace(' ', '-') +'.webp')

In [664]:
top_10

,Company,Number of Positive reviews,reviews_total,positive business outlook %,company logo image name
85,ServiceNow,347,500,0.694000,servicenow.webp
68,NVIDIA,253,365,0.693151,nvidia.webp
52,Intuit,648,1013,0.639684,intuit.webp
73,Palo Alto Networks,179,288,0.621528,palo-alto-networks.webp
4,Adobe,331,550,0.601818,adobe.webp
60,Mastercard,228,399,0.571429,mastercard.webp
22,Boston Scientific,260,486,0.534979,boston-scientific.webp
92,Texas Instruments,219,431,0.508121,texas-instruments.webp
49,Honeywell,754,1497,0.503674,honeywell.webp
41,Eli Lilly,182,369,0.493225,eli-lilly.webp


## Bottom 10

Bottom 10: Bottom 10 companies on % of employees with positive business outlook
    
Columns: Company, % of employees with positive business outlook, sample size of employees, company logo image link


In [665]:
#bottom_10 = df[df.review_date >= one_year_ago]
bottom_10 = df[df.business_outlook=='POSITIVE'].groupby('Company')['review_id'].count().reset_index()

bottom_10.columns = ['Company', 'Number of Positive reviews']

bottom_10 = bottom_10.merge(number_of_reviews, on='Company', how='left')

bottom_10['positive business outlook %'] = bottom_10['Number of Positive reviews']/bottom_10['reviews_total']
bottom_10 = bottom_10[bottom_10.reviews_total >=quantile_25]

bottom_10 = bottom_10.sort_values(by='positive business outlook %', ascending=True).head(10)

In [666]:
bottom_10['company logo image name'] = bottom_10['Company'].apply(lambda x: x.lower().replace('\'', '').replace('(facebook)', '').replace('&', '').replace(' ', '-') +'.webp')

In [667]:
bottom_10

,Company,Number of Positive reviews,reviews_total,positive business outlook %,company logo image name
61,McDonald,2509,13485,0.186059,mcdonald.webp
95,Union Pacific Corporation,75,357,0.210084,union-pacific-corporation.webp
25,CVS Health,1501,7067,0.212396,cvs-health.webp
101,Walmart,3888,17431,0.223051,walmart.webp
86,Starbucks,2635,10907,0.241588,starbucks.webp
103,Wells Fargo,1362,5461,0.249405,wells-fargo.webp
44,Fiserv,252,946,0.266385,fiserv.webp
47,Goldman Sachs,459,1595,0.287774,goldman-sachs.webp
63,Meta Platforms (Facebook),741,2562,0.289227,meta-platforms-.webp
102,Walt Disney,553,1907,0.289984,walt-disney.webp


## Overall Change

Overall Change: Top 10 & bottom 10 companies with most positive and negative year-on-year change in % of employees with positive business outlook

Columns: Company,  (2022-2023) % of employees with positive business outlook, (2023-2024) % of employees with positive business outlook, sample size of employees (2022/23), sample size of employees (2023/24)


In [752]:
last_year = df[df.review_date >= one_year_ago]

In [753]:
last_year.review_date.min()

Timestamp('2023-03-24 00:00:00')

In [754]:
last_year.review_date.max()

Timestamp('2024-03-24 00:00:00')

In [755]:
last_year = last_year[last_year.business_outlook=='POSITIVE'].groupby('Company')['review_id'].count().reset_index()

last_year.columns = ['Company', 'Number of Positive reviews']

last_year = last_year.merge(number_of_reviews_last_year, on='Company', how='left')

last_year['positive business outlook %'] = last_year['Number of Positive reviews']/last_year['reviews_total']

#last_year = last_year[last_year.reviews_total >=220]
last_year.columns = ['Company', 'Number of Positive reviews (2023-2024)', 
                     'reviews_total (2023-2024)', 'positive business outlook % (2023-2024)']
last_year_average_percent = last_year['positive business outlook % (2023-2024)']
last_year['average_percent (2023-2024)'] = last_year['positive business outlook % (2023-2024)'].mean()
last_year

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024)
0,AMD,88,171,0.514620,0.388784
1,AT&T,660,2522,0.261697,0.388784
2,AbbVie,131,354,0.370056,0.388784
3,Abbott Laboratories,178,513,0.346979,0.388784
4,Adobe,174,301,0.578073,0.388784
...,...,...,...,...,...
99,Vertex Pharmaceuticals,57,92,0.619565,0.388784
100,Visa,108,270,0.400000,0.388784
101,Walmart,2010,9813,0.204830,0.388784
102,Walt Disney,314,1120,0.280357,0.388784


In [760]:
two_years_ago = df[df.review_date < one_year_ago]


In [761]:
two_years_ago.review_date.min()

Timestamp('2022-03-24 00:00:00')

In [762]:
two_years_ago.review_date.max()

Timestamp('2023-03-23 00:00:00')

In [763]:

two_years_ago = two_years_ago[two_years_ago.business_outlook=='POSITIVE'].groupby('Company')['review_id'].count().reset_index()

two_years_ago.columns = ['Company', 'Number of Positive reviews']

two_years_ago = two_years_ago.merge(number_of_reviews_two_years_ago, on='Company', how='left')

two_years_ago['positive business outlook %'] = two_years_ago['Number of Positive reviews']/two_years_ago['reviews_total']

#two_years_ago = two_years_ago[two_years_ago.reviews_total >=220]
two_years_ago.columns = ['Company', 'Number of Positive reviews (2022-2023)', 
                     'reviews_total (2022-2023)', 'positive business outlook % (2022-2023)']
two_years_ago['average_percent (2022-2023)'] = two_years_ago['positive business outlook % (2022-2023)'].mean()

two_years_ago

,Company,Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023)
0,AMD,49,89,0.550562,0.448821
1,AT&T,696,1900,0.366316,0.448821
2,AbbVie,98,266,0.368421,0.448821
3,Abbott Laboratories,160,377,0.424403,0.448821
4,Adobe,157,249,0.630522,0.448821
...,...,...,...,...,...
99,Vertex Pharmaceuticals,81,117,0.692308,0.448821
100,Visa,93,195,0.476923,0.448821
101,Walmart,1878,7618,0.246521,0.448821
102,Walt Disney,239,787,0.303685,0.448821


In [764]:
overall_change = last_year.merge(two_years_ago, on='Company', how='left')

In [765]:
overall_change['change'] = overall_change['positive business outlook % (2023-2024)'] - overall_change['positive business outlook % (2022-2023)']

In [766]:
overall_change.dropna(subset=['change'], inplace=True)

In [767]:
overall_change = overall_change[(overall_change['reviews_total (2023-2024)'] > 146.375) & (overall_change['reviews_total (2022-2023)'] > 220)]


In [768]:
overall_change

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024),Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023),change
1,AT&T,660,2522,0.261697,0.388784,696,1900,0.366316,0.448821,-0.104619
2,AbbVie,131,354,0.370056,0.388784,98,266,0.368421,0.448821,0.001635
3,Abbott Laboratories,178,513,0.346979,0.388784,160,377,0.424403,0.448821,-0.077425
4,Adobe,174,301,0.578073,0.388784,157,249,0.630522,0.448821,-0.052449
6,Alphabet (Google),929,2963,0.313534,0.388784,731,1810,0.403867,0.448821,-0.090334
7,Amazon,3068,12276,0.249919,0.388784,4030,11352,0.355004,0.448821,-0.105085
8,American Express,230,651,0.353303,0.388784,176,401,0.438903,0.448821,-0.085600
12,Apple,1013,2496,0.405849,0.388784,848,1817,0.466703,0.448821,-0.060854
15,Automatic Data Processing,457,1165,0.392275,0.388784,370,848,0.436321,0.448821,-0.044046
16,Bank of America,708,2196,0.322404,0.388784,676,1803,0.374931,0.448821,-0.052526


In [769]:
top_10_overall_change = overall_change.sort_values(by='change', ascending=False).head(10)
top_10_overall_change['company logo image name'] = top_10_overall_change['Company'].apply(lambda x: x.lower().replace('\'', '').replace('(facebook)', '').replace('&', '').replace(' ', '-') +'.webp')
top_10_overall_change

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024),Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023),change,company logo image name
63,Meta Platforms (Facebook),482,1487,0.324143,0.388784,259,1075,0.240930,0.448821,0.083212,meta-platforms-.webp
45,General Electric,113,363,0.311295,0.388784,71,257,0.276265,0.448821,0.035030,general-electric.webp
44,Fiserv,148,550,0.269091,0.388784,104,396,0.262626,0.448821,0.006465,fiserv.webp
51,Intel,393,1136,0.345951,0.388784,300,879,0.341297,0.448821,0.004654,intel.webp
54,JPMorgan Chase,1106,2747,0.402621,0.388784,808,2028,0.398422,0.448821,0.004199,jpmorgan-chase.webp
84,Salesforce,563,1243,0.452936,0.388784,393,871,0.451206,0.448821,0.001731,salesforce.webp
2,AbbVie,131,354,0.370056,0.388784,98,266,0.368421,0.448821,0.001635,abbvie.webp
81,Raytheon Technologies,157,479,0.327766,0.388784,92,278,0.330935,0.448821,-0.003169,raytheon-technologies.webp
62,Merck,168,351,0.478632,0.388784,128,257,0.498054,0.448821,-0.019422,merck.webp
90,TJX Companies,137,350,0.391429,0.388784,118,286,0.412587,0.448821,-0.021159,tjx-companies.webp


In [770]:
bottom_10_overall_change = overall_change.sort_values(by='change', ascending=True).head(10)
bottom_10_overall_change['company logo image name'] = bottom_10_overall_change['Company'].apply(lambda x: x.lower().replace('\'', '').replace('(facebook)', '').replace('&', '').replace(' ', '-') +'.webp')
bottom_10_overall_change

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024),Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023),change,company logo image name
75,Pfizer,145,491,0.295316,0.388784,147,319,0.460815,0.448821,-0.165499,pfizer.webp
39,Dell,377,1162,0.324441,0.388784,358,744,0.481183,0.448821,-0.156742,dell.webp
93,Thermo Fisher Scientific,295,884,0.333710,0.388784,262,538,0.486989,0.448821,-0.153278,thermo-fisher-scientific.webp
28,Charles Schwab,289,847,0.341204,0.388784,252,546,0.461538,0.448821,-0.120334,charles-schwab.webp
30,Cigna,180,537,0.335196,0.388784,176,395,0.445570,0.448821,-0.110374,cigna.webp
23,Bristol-Myers Squibb,129,402,0.320896,0.388784,123,286,0.430070,0.448821,-0.109174,bristol-myers-squibb.webp
7,Amazon,3068,12276,0.249919,0.388784,4030,11352,0.355004,0.448821,-0.105085,amazon.webp
1,AT&T,660,2522,0.261697,0.388784,696,1900,0.366316,0.448821,-0.104619,att.webp
31,Cisco,354,846,0.418440,0.388784,322,616,0.522727,0.448821,-0.104288,cisco.webp
50,IBM,465,1499,0.310207,0.388784,507,1225,0.413878,0.448821,-0.103671,ibm.webp


## Tech Giants

Tech Giants: Top 10 and bottom 10 tech companies on % of employees with positive business outlook

Columns: Tech company, % of employees with positive business outlook, sample size of employees, company logo image link

In [702]:
tech_giants = df[df.Industry == 'Tech']

In [703]:
tech_giants = tech_giants[tech_giants.business_outlook=='POSITIVE'].groupby('Company')['review_id'].count().reset_index()

tech_giants.columns = ['Company', 'Number of Positive reviews']

tech_giants = tech_giants.merge(number_of_reviews, on='Company', how='left')

tech_giants['positive business outlook %'] = tech_giants['Number of Positive reviews']/tech_giants['reviews_total']

tech_giants = tech_giants[tech_giants.reviews_total >=quantile_25]
tech_giants_top = tech_giants.sort_values(by='positive business outlook %', ascending=False).head(10)

tech_giants_top['company logo image name'] = tech_giants_top['Company'].apply(lambda x: x.lower().replace(' ', '-') +'.webp')

tech_giants_top

,Company,Number of Positive reviews,reviews_total,positive business outlook %,company logo image name
31,ServiceNow,347,500,0.694000,servicenow.webp
25,NVIDIA,253,365,0.693151,nvidia.webp
19,Intuit,648,1013,0.639684,intuit.webp
28,Palo Alto Networks,179,288,0.621528,palo-alto-networks.webp
1,Adobe,331,550,0.601818,adobe.webp
34,Texas Instruments,219,431,0.508121,texas-instruments.webp
7,Applied Materials,217,453,0.479029,applied-materials.webp
24,Microsoft,1723,3603,0.478213,microsoft.webp
13,Cisco,676,1462,0.462380,cisco.webp
30,Salesforce,956,2114,0.452223,salesforce.webp


In [704]:
tech_giants_bottom = tech_giants.sort_values(by='positive business outlook %', ascending=True).head(10)

tech_giants_bottom['company logo image name'] = tech_giants_bottom['Company'].apply(lambda x: x.lower().replace(' ', '-') +'.webp')

tech_giants_bottom

,Company,Number of Positive reviews,reviews_total,positive business outlook %,company logo image name
16,Fiserv,252,946,0.266385,fiserv.webp
22,Meta Platforms (Facebook),741,2562,0.289227,meta-platforms-(facebook).webp
4,Amazon,7098,23628,0.300406,amazon.webp
35,Uber,476,1531,0.310908,uber.webp
29,QUALCOMM,135,425,0.317647,qualcomm.webp
27,Oracle,880,2765,0.318264,oracle.webp
18,Intel,693,2015,0.343921,intel.webp
3,Alphabet (Google),1660,4773,0.347790,alphabet-(google).webp
17,IBM,972,2724,0.356828,ibm.webp
14,Dell,735,1906,0.385624,dell.webp


## Tech Change

Tech Change: Top 10 & bottom 10 tech companies with most positive and negative year-on-year change in % of employees with positive business outlook

Columns: Tech Company,  (2022-2023) % of employees with positive business outlook, (2023-2024) % of employees with positive business outlook, sample size of employees (2022/23), sample size of employees (2023/24)

In [771]:
tech_change = df[df.Industry == 'Tech']

In [772]:
tech_change_last_year = tech_change[tech_change.review_date >= one_year_ago]

In [773]:
tech_change_last_year = tech_change_last_year[tech_change_last_year.business_outlook=='POSITIVE'].groupby('Company')['review_id'].count().reset_index()

tech_change_last_year.columns = ['Company', 'Number of Positive reviews']

tech_change_last_year = tech_change_last_year.merge(number_of_reviews_last_year, on='Company', how='left')

tech_change_last_year['positive business outlook %'] = tech_change_last_year['Number of Positive reviews']/tech_change_last_year['reviews_total']

#last_year = last_year[last_year.reviews_total >=220]
tech_change_last_year.columns = ['Company', 'Number of Positive reviews (2023-2024)', 
                     'reviews_total (2023-2024)', 'positive business outlook % (2023-2024)']
tech_change_last_year['average_percent (2023-2024)'] = tech_change_last_year['positive business outlook % (2023-2024)'].mean()

tech_change_last_year

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024)
0,AMD,88,171,0.514620,0.423077
1,Adobe,174,301,0.578073,0.423077
2,Airbnb,19,61,0.311475,0.423077
3,Alphabet (Google),929,2963,0.313534,0.423077
4,Amazon,3068,12276,0.249919,0.423077
5,Analog Devices,37,125,0.296000,0.423077
6,Apple,1013,2496,0.405849,0.423077
7,Applied Materials,123,250,0.492000,0.423077
8,Arista Networks,25,37,0.675676,0.423077
9,Automatic Data Processing,457,1165,0.392275,0.423077


In [775]:
tech_change_two_years_ago = tech_change[tech_change.review_date < one_year_ago]


In [776]:
tech_change_two_years_ago = tech_change_two_years_ago[tech_change_two_years_ago.business_outlook=='POSITIVE'].groupby('Company')['review_id'].count().reset_index()

tech_change_two_years_ago.columns = ['Company', 'Number of Positive reviews']

tech_change_two_years_ago = tech_change_two_years_ago.merge(number_of_reviews_two_years_ago, on='Company', how='left')

tech_change_two_years_ago['positive business outlook %'] = tech_change_two_years_ago['Number of Positive reviews']/tech_change_two_years_ago['reviews_total']

#two_years_ago = two_years_ago[two_years_ago.reviews_total >=220]
tech_change_two_years_ago.columns = ['Company', 'Number of Positive reviews (2022-2023)', 
                     'reviews_total (2022-2023)', 'positive business outlook % (2022-2023)']
tech_change_two_years_ago['average_percent (2022-2023)'] = tech_change_two_years_ago['positive business outlook % (2022-2023)'].mean()

tech_change_two_years_ago

,Company,Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023)
0,AMD,49,89,0.550562,0.485461
1,Adobe,157,249,0.630522,0.485461
2,Airbnb,25,56,0.446429,0.485461
3,Alphabet (Google),731,1810,0.403867,0.485461
4,Amazon,4030,11352,0.355004,0.485461
5,Analog Devices,45,86,0.523256,0.485461
6,Apple,848,1817,0.466703,0.485461
7,Applied Materials,94,203,0.463054,0.485461
8,Arista Networks,15,23,0.652174,0.485461
9,Automatic Data Processing,370,848,0.436321,0.485461


In [777]:
overall_tech_change = tech_change_last_year.merge(tech_change_two_years_ago, on='Company', how='left')

overall_tech_change['change'] = overall_tech_change['positive business outlook % (2023-2024)'] - overall_tech_change['positive business outlook % (2022-2023)']

overall_tech_change.dropna(subset=['change'], inplace=True)

#overall_tech_change = overall_tech_change[(overall_tech_change['reviews_total (2023-2024)'] > 146.375) & (overall_tech_change['reviews_total (2022-2023)'] > 220)]


overall_tech_change

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024),Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023),change
0,AMD,88,171,0.514620,0.423077,49,89,0.550562,0.485461,-0.035942
1,Adobe,174,301,0.578073,0.423077,157,249,0.630522,0.485461,-0.052449
2,Airbnb,19,61,0.311475,0.423077,25,56,0.446429,0.485461,-0.134953
3,Alphabet (Google),929,2963,0.313534,0.423077,731,1810,0.403867,0.485461,-0.090334
4,Amazon,3068,12276,0.249919,0.423077,4030,11352,0.355004,0.485461,-0.105085
5,Analog Devices,37,125,0.296000,0.423077,45,86,0.523256,0.485461,-0.227256
6,Apple,1013,2496,0.405849,0.423077,848,1817,0.466703,0.485461,-0.060854
7,Applied Materials,123,250,0.492000,0.423077,94,203,0.463054,0.485461,0.028946
8,Arista Networks,25,37,0.675676,0.423077,15,23,0.652174,0.485461,0.023502
9,Automatic Data Processing,457,1165,0.392275,0.423077,370,848,0.436321,0.485461,-0.044046


In [778]:
top_overall_tech_change = overall_tech_change.sort_values(by='change', ascending=False).head(10)
top_overall_tech_change['company logo image name'] = top_overall_tech_change['Company'].apply(lambda x: x.lower().replace('\'', '').replace('(facebook)', '').replace('&', '').replace(' ', '-') +'.webp')
top_overall_tech_change

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024),Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023),change,company logo image name
22,Meta Platforms (Facebook),482,1487,0.324143,0.423077,259,1075,0.240930,0.485461,0.083212,meta-platforms-.webp
32,Synopsys,45,84,0.535714,0.423077,26,53,0.490566,0.485461,0.045148,synopsys.webp
7,Applied Materials,123,250,0.492000,0.423077,94,203,0.463054,0.485461,0.028946,applied-materials.webp
8,Arista Networks,25,37,0.675676,0.423077,15,23,0.652174,0.485461,0.023502,arista-networks.webp
12,Cadence Design Systems,44,85,0.517647,0.423077,24,48,0.500000,0.485461,0.017647,cadence-design-systems.webp
16,Fiserv,148,550,0.269091,0.423077,104,396,0.262626,0.485461,0.006465,fiserv.webp
18,Intel,393,1136,0.345951,0.423077,300,879,0.341297,0.485461,0.004654,intel.webp
30,Salesforce,563,1243,0.452936,0.423077,393,871,0.451206,0.485461,0.001731,salesforce.webp
23,Micron Technology,80,201,0.398010,0.423077,91,218,0.417431,0.485461,-0.019421,micron-technology.webp
21,Lam Research,61,147,0.414966,0.423077,53,122,0.434426,0.485461,-0.019460,lam-research.webp


In [779]:
bottom_overall_tech_change = overall_tech_change.sort_values(by='change', ascending=True).head(10)
bottom_overall_tech_change['company logo image name'] = bottom_overall_tech_change['Company'].apply(lambda x: x.lower().replace('\'', '').replace('(facebook)', '').replace('&', '').replace(' ', '-') +'.webp')
bottom_overall_tech_change

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024),Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023),change,company logo image name
5,Analog Devices,37,125,0.296000,0.423077,45,86,0.523256,0.485461,-0.227256,analog-devices.webp
25,NVIDIA,119,195,0.610256,0.423077,134,170,0.788235,0.485461,-0.177979,nvidia.webp
34,Texas Instruments,110,253,0.434783,0.423077,109,178,0.612360,0.485461,-0.177577,texas-instruments.webp
15,Equinix,38,77,0.493506,0.423077,43,65,0.661538,0.485461,-0.168032,equinix.webp
14,Dell,377,1162,0.324441,0.423077,358,744,0.481183,0.485461,-0.156742,dell.webp
29,QUALCOMM,81,294,0.275510,0.423077,54,131,0.412214,0.485461,-0.136704,qualcomm.webp
2,Airbnb,19,61,0.311475,0.423077,25,56,0.446429,0.485461,-0.134953,airbnb.webp
4,Amazon,3068,12276,0.249919,0.423077,4030,11352,0.355004,0.485461,-0.105085,amazon.webp
13,Cisco,354,846,0.418440,0.423077,322,616,0.522727,0.485461,-0.104288,cisco.webp
17,IBM,465,1499,0.310207,0.423077,507,1225,0.413878,0.485461,-0.103671,ibm.webp


## Industries

Industries: Top company and bottom company per industry on % of employees with positive business outlook. Include the US average %.

Columns: Industry, Top company, % of employees with positive business outlook, Bottom company, % of employees with positive business outlook, company logo image link


In [780]:
industries_count = seed_list.groupby('Industry')['Company'].count().reset_index().sort_values(by='Company', ascending=False)
industries_count = industries_count[industries_count.Company >2]
industries_list = industries_count.Industry.to_list()
industries_list.remove('Tech')

In [781]:
industries_list.remove('Oil&Gas and Energy',)

In [782]:
industries_list

['Pharmaceuticals, Healthcare, Medical devices',
 'Banks & Financial services',
 'Food & Beverages',
 'Aircraft manufacturers & Aerospace',
 '🛍️ Retail',
 'Machinery manufacturing',
 '📡 Telecommunication']

In [783]:
industries = overall_change.merge(seed_list, on='Company', how='left')

In [784]:
industries = industries[industries.Industry.isin(industries_list)]

In [785]:
industries

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024),Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023),change,Industry
0,AT&T,660,2522,0.261697,0.388784,696,1900,0.366316,0.448821,-0.104619,📡 Telecommunication
1,AbbVie,131,354,0.370056,0.388784,98,266,0.368421,0.448821,0.001635,"Pharmaceuticals, Healthcare, Medical devices"
2,Abbott Laboratories,178,513,0.346979,0.388784,160,377,0.424403,0.448821,-0.077425,"Pharmaceuticals, Healthcare, Medical devices"
6,American Express,230,651,0.353303,0.388784,176,401,0.438903,0.448821,-0.085600,Banks & Financial services
9,Bank of America,708,2196,0.322404,0.388784,676,1803,0.374931,0.448821,-0.052526,Banks & Financial services
10,Boeing,317,900,0.352222,0.388784,335,870,0.385057,0.448821,-0.032835,Aircraft manufacturers & Aerospace
11,Bristol-Myers Squibb,129,402,0.320896,0.388784,123,286,0.430070,0.448821,-0.109174,"Pharmaceuticals, Healthcare, Medical devices"
12,CVS Health,805,3977,0.202414,0.388784,696,3090,0.225243,0.448821,-0.022829,"Pharmaceuticals, Healthcare, Medical devices"
13,Caterpillar,127,300,0.423333,0.388784,106,225,0.471111,0.448821,-0.047778,Machinery manufacturing
14,Charles Schwab,289,847,0.341204,0.388784,252,546,0.461538,0.448821,-0.120334,Banks & Financial services


In [786]:
highest_change = industries.groupby('Industry').apply(lambda x: x.nlargest(1, 'change'))
lowest_change = industries.groupby('Industry').apply(lambda x: x.nsmallest(1, 'change'))

In [789]:
result_industries = pd.concat([highest_change, lowest_change])

result_industries.reset_index(drop=True, inplace=True)

result_industries = result_industries.sort_values(by=['Industry', 'change'], ascending=True)
result_industries['company logo image name'] = result_industries['Company'].apply(lambda x: x.lower().replace('\'', '').replace('(facebook)', '').replace('&', '').replace(' ', '-') +'.webp')

result_industries

,Company,Number of Positive reviews (2023-2024),reviews_total (2023-2024),positive business outlook % (2023-2024),average_percent (2023-2024),Number of Positive reviews (2022-2023),reviews_total (2022-2023),positive business outlook % (2022-2023),average_percent (2022-2023),change,Industry,company logo image name
7,Honeywell,329,725,0.453793,0.388784,425,772,0.550518,0.448821,-0.096725,Aircraft manufacturers & Aerospace,honeywell.webp
0,Raytheon Technologies,157,479,0.327766,0.388784,92,278,0.330935,0.448821,-0.003169,Aircraft manufacturers & Aerospace,raytheon-technologies.webp
8,Charles Schwab,289,847,0.341204,0.388784,252,546,0.461538,0.448821,-0.120334,Banks & Financial services,charles-schwab.webp
1,JPMorgan Chase,1106,2747,0.402621,0.388784,808,2028,0.398422,0.448821,0.004199,Banks & Financial services,jpmorgan-chase.webp
9,Starbucks,1500,6880,0.218023,0.388784,1135,4027,0.281848,0.448821,-0.063824,Food & Beverages,starbucks.webp
2,McDonald,1395,8271,0.168662,0.388784,1114,5214,0.213656,0.448821,-0.044994,Food & Beverages,mcdonald.webp
10,Caterpillar,127,300,0.423333,0.388784,106,225,0.471111,0.448821,-0.047778,Machinery manufacturing,caterpillar.webp
3,General Electric,113,363,0.311295,0.388784,71,257,0.276265,0.448821,0.035030,Machinery manufacturing,general-electric.webp
11,Pfizer,145,491,0.295316,0.388784,147,319,0.460815,0.448821,-0.165499,"Pharmaceuticals, Healthcare, Medical devices",pfizer.webp
4,AbbVie,131,354,0.370056,0.388784,98,266,0.368421,0.448821,0.001635,"Pharmaceuticals, Healthcare, Medical devices",abbvie.webp


## Top State

Top State: Top company state on % of employees with positive business outlook
    
Columns: Top company, % of employees with positive business outlook


In [804]:
top_state = df[df.business_outlook=='POSITIVE'].groupby(['State','Company'])['review_id'].count().reset_index()

In [805]:
top_state.columns = ['State', 'Company', 'Number of Positive reviews']

In [802]:
reviews_number_state = df.groupby('State')['review_id'].count().reset_index()
reviews_number_state.columns = ['State', 'reviews_total']
reviews_number_state

,State,reviews_total
0,Alabama,1731
1,Alaska,164
2,Arizona,5386
3,Arkansas,1535
4,California,31142
5,Colorado,3181
6,Connecticut,1573
7,Delaware,957
8,District of Columbia,1230
9,Florida,12343


In [808]:
top_state = top_state.merge(reviews_number_state, on='State', how='left')

top_state['positive business outlook %'] = top_state['Number of Positive reviews']/top_state['reviews_total']




In [810]:
top_state = top_state.groupby('State').apply(lambda x: x.nlargest(1, 'positive business outlook %'))
top_state.reset_index(drop=True, inplace=True)

In [814]:
top_state['company logo image name'] = top_state['Company'].apply(lambda x: x.lower().replace('\'', '').replace('(facebook)', '').replace('&', '').replace(' ', '-') +'.webp')

In [816]:
len(top_state)

51

In [815]:
top_state

,State,Company,Number of Positive reviews,reviews_total,positive business outlook %,company logo image name
0,Alabama,Walmart,88,1731,0.050838,walmart.webp
1,Alaska,Home Depot,8,164,0.048780,home-depot.webp
2,Arizona,Amazon,245,5386,0.045488,amazon.webp
3,Arkansas,Walmart,315,1535,0.205212,walmart.webp
4,California,Amazon,932,31142,0.029927,amazon.webp
5,Colorado,Amazon,118,3181,0.037095,amazon.webp
6,Connecticut,CVS Health,73,1573,0.046408,cvs-health.webp
7,Delaware,JPMorgan Chase,148,957,0.154650,jpmorgan-chase.webp
8,District of Columbia,Amazon,56,1230,0.045528,amazon.webp
9,Florida,Walt Disney,293,12343,0.023738,walt-disney.webp


## Bottom State

Bottom State: Top company state on % of employees with positive business outlook
    
Columns: Bottom company, % of employees with positive business outlook


In [817]:
bottom_state = df[df.business_outlook=='POSITIVE'].groupby(['State','Company'])['review_id'].count().reset_index()

bottom_state.columns = ['State', 'Company', 'Number of Positive reviews']

bottom_state = bottom_state.merge(reviews_number_state, on='State', how='left')

bottom_state['positive business outlook %'] = bottom_state['Number of Positive reviews']/bottom_state['reviews_total']

bottom_state = bottom_state.groupby('State').apply(lambda x: x.nsmallest(1, 'positive business outlook %'))
bottom_state.reset_index(drop=True, inplace=True)

bottom_state['company logo image name'] = bottom_state['Company'].apply(lambda x: x.lower().replace('\'', '').replace('(facebook)', '').replace('&', '').replace(' ', '-') +'.webp')

In [818]:
len(bottom_state)

51

In [819]:
bottom_state

,State,Company,Number of Positive reviews,reviews_total,positive business outlook %,company logo image name
0,Alabama,Abbott Laboratories,1,1731,0.000578,abbott-laboratories.webp
1,Alaska,AT&T,1,164,0.006098,att.webp
2,Arizona,AbbVie,1,5386,0.000186,abbvie.webp
3,Arkansas,Charles Schwab,1,1535,0.000651,charles-schwab.webp
4,California,American Tower,1,31142,0.000032,american-tower.webp
5,Colorado,Airbnb,1,3181,0.000314,airbnb.webp
6,Connecticut,Alphabet (Google),1,1573,0.000636,alphabet-(google).webp
7,Delaware,AT&T,1,957,0.001045,att.webp
8,District of Columbia,Abbott Laboratories,1,1230,0.000813,abbott-laboratories.webp
9,Florida,BlackRock,1,12343,0.000081,blackrock.webp


## California

California: Top 10 and bottom 10 companies from employees based in California, based on % of employees with positive business outlook

Columns: Top company, % of employees with positive business outlook

In [820]:
california = df[df.State=='California']

In [821]:
reviews_number_california = df.groupby('Company')['review_id'].count().reset_index()
reviews_number_california.columns = ['Company', 'reviews_total']
reviews_number_california

,Company,reviews_total
0,AMD,260
1,AT&T,4422
2,AbbVie,620
3,Abbott Laboratories,890
4,Adobe,550
...,...,...
99,Vertex Pharmaceuticals,209
100,Visa,465
101,Walmart,17431
102,Walt Disney,1907


In [822]:
quantile_25 = reviews_number_california.reviews_total.quantile(0.25)
quantile_25

283.25

In [823]:
california = california[california.business_outlook=='POSITIVE'].groupby('Company')['review_id'].count().reset_index()
california.columns = ['Company', 'Number of Positive reviews']
california = california.merge(reviews_number_california, on='Company', how='left')
california['positive business outlook %'] = california['Number of Positive reviews']/california['reviews_total']
california = california[california.reviews_total >=quantile_25]
california

,Company,Number of Positive reviews,reviews_total,positive business outlook %
1,AT&T,150,4422,0.033921
2,AbbVie,22,620,0.035484
3,Abbott Laboratories,69,890,0.077528
4,Adobe,144,550,0.261818
6,Alphabet (Google),830,4773,0.173895
...,...,...,...,...
92,Verizon,59,2907,0.020296
94,Visa,73,465,0.156989
95,Walmart,232,17431,0.013310
96,Walt Disney,177,1907,0.092816


In [824]:
top_california = california.sort_values(by='positive business outlook %', ascending=False).head(10)

top_california['company logo image name'] = top_california['Company'].apply(lambda x: x.lower().replace(' ', '-') +'.webp')

top_california

,Company,Number of Positive reviews,reviews_total,positive business outlook %,company logo image name
65,NVIDIA,193,365,0.528767,nvidia.webp
80,ServiceNow,187,500,0.374000,servicenow.webp
70,Palo Alto Networks,107,288,0.371528,palo-alto-networks.webp
76,QUALCOMM,117,425,0.275294,qualcomm.webp
49,Intuit,274,1013,0.270484,intuit.webp
4,Adobe,144,550,0.261818,adobe.webp
10,Amgen,76,308,0.246753,amgen.webp
86,Tesla,407,1841,0.221076,tesla.webp
30,Cisco,310,1462,0.212038,cisco.webp
13,Applied Materials,89,453,0.196468,applied-materials.webp


In [825]:
bottom_california = california.sort_values(by='positive business outlook %', ascending=True).head(10)

bottom_california['company logo image name'] = bottom_california['Company'].apply(lambda x: x.lower().replace(' ', '-') +'.webp')

bottom_california

,Company,Number of Positive reviews,reviews_total,positive business outlook %,company logo image name
41,Fiserv,3,946,0.003171,fiserv.webp
26,Caterpillar,2,525,0.003810,caterpillar.webp
8,American Express,6,1052,0.005703,american-express.webp
36,Deere & Company,3,448,0.006696,deere-&-company.webp
42,General Electric,5,620,0.008065,general-electric.webp
44,Goldman Sachs,15,1595,0.009404,goldman-sachs.webp
29,Cigna,9,932,0.009657,cigna.webp
73,Procter & Gamble,7,647,0.010819,procter-&-gamble.webp
32,Coca-Cola,5,427,0.011710,coca-cola.webp
74,Progressive,21,1641,0.012797,progressive.webp


# Create an Excel file

In [827]:
# Create an Excel writer object

writer = pd.ExcelWriter('The Companies Where Employee Confidence is Highest and Lowest.xlsx')

In [828]:
top_10.to_excel(writer, sheet_name='Top 10', index=False)
bottom_10.to_excel(writer, sheet_name='Bottom 10', index=False)

In [829]:
top_10_overall_change.to_excel(writer, sheet_name='Overall Change', startrow=0, startcol=0, index=False)
bottom_10_overall_change.to_excel(writer, sheet_name='Overall Change', startrow=0, 
                   startcol=len(top_10_overall_change.columns) + 2, index=False)

In [830]:
tech_giants_top.to_excel(writer, sheet_name='Tech Giants', startrow=0, startcol=0, index=False)
tech_giants_bottom.to_excel(writer, sheet_name='Tech Giants', startrow=0, 
                   startcol=len(tech_giants_top.columns) + 2, index=False)

In [831]:
top_overall_tech_change.to_excel(writer, sheet_name='Tech Change', startrow=0, startcol=0, index=False)
bottom_overall_tech_change.to_excel(writer, sheet_name='Tech Change', startrow=0, 
                   startcol=len(top_overall_tech_change.columns) + 2, index=False)

In [832]:
result_industries.to_excel(writer, sheet_name='Industries', index=False)

In [833]:
top_state.to_excel(writer, sheet_name='Top State', index=False)
bottom_state.to_excel(writer, sheet_name='Bottom State', index=False)

In [834]:
top_california.to_excel(writer, sheet_name='California', startrow=0, startcol=0, index=False)
bottom_california.to_excel(writer, sheet_name='California', startrow=0, 
                   startcol=len(top_california.columns) + 2, index=False)

In [835]:
df.to_excel(writer, sheet_name='Raw Data', index=False)


In [836]:
# Save the Excel file
writer.save()